In [22]:
import argparse
from pathlib import Path
from fastai.imagito.utils import *
from fastai.vision.models.xresnet2 import xresnet50_2
import torch
import math
from operator import itemgetter

from sklearn.model_selection import ParameterGrid
from fastai.imagito.utils import update_batch_size

to_grid = lambda p: update_batch_size(ParameterGrid(p))

In [13]:
def load_model(model_dir, size, woof, bs, sample):
#     model_params = pickle_load(model_dir + '/params.pkl')
#     if model_params['size'] != size:
#         print('Warning: Model was trained on image size=%d but is being used to predict for size=%d'
#               % (model_params['size'], size))

    m = xresnet50_2
    #print(model_params)
    data = get_data(size, woof, bs, sample, shuffle_train=False, flip_lr_p=0.0)
    l = Learner(data, m(c_out=10), path=model_dir)
    l.load('woof_classif')
    return l, data

In [ ]:
!mv woof_classif.pth

In [4]:
mkdir models

In [5]:
mv woof_classif.pth models

In [15]:
ls models

woof_classif.pth


In [19]:
model, data = load_model('.', 128, True, 256, 1.)

In [20]:
predictions_t, targets_t, loss_t = model.get_preds(ds_type=DatasetType.Train, with_loss=True)

In [21]:
to_arr = lambda x: x.numpy()

In [27]:
paths = list(map(lambda x: x.name,data.items))

In [33]:
df = pd.DataFrame(to_arr(loss_t), columns=['loss'])
df.index = paths

In [43]:
df = df.rename_axis('paths').sort_values('loss', ascending=False).reset_index()

In [47]:
df.head()

,paths,loss
0,n02086240_7862.JPEG,11.440737
1,n02089973_2860.JPEG,10.591484
2,n02088364_12006.JPEG,8.879482
3,n02086240_7549.JPEG,8.092752
4,n02093754_5276.JPEG,7.432945


In [19]:
from sklearn.model_selection import ParameterGrid
from fastai.imagito.utils import update_batch_size
from fastai.imagito.classes import ClassUtils

to_grid = lambda p: update_batch_size(ParameterGrid(p))

In [21]:
!git pull

Already up-to-date.


In [20]:
ClassUtils.from_indices()

['n02089973',
 'n02086240',
 'n02105641',
 'n02087394',
 'n02099601',
 'n02115641',
 'n02093754',
 'n02111889',
 'n02096294',
 'n02088364']

In [33]:
len(GRID)

18

In [41]:
run = [dict(opt='adam', stem1=32, stem2=32)]

In [42]:
from sklearn.utils import shuffle

In [35]:
GRID = to_grid({'stem1': [4, 16, 32, 48],
'stem2': [4, 16, 32, 48],
'opt': ['adam', 'sgd', 'rms']})

In [46]:
GRID = [x for x in shuffle(GRID) if x not in run]

In [47]:
len(GRID)

47

In [62]:
pickle_save(GRID, 'grid.pkl')

In [64]:
ARCH = 'xresnet18'

In [51]:
IGNORE = 'distillation'

In [56]:
from tqdm import *
from fastai.imagito.train_imagito import main

In [ ]:
for g in tqdm_notebook(GRID):
    for name, strat in STRAT2PARAMS_V2.items():
        if name in IGNORE:
            continue
        else:
            print(strat, g)
            main(arch=ARCH, **strat, **g)
        

{'classes': None, 'sample': 0.1} {'opt': 'sgd', 'stem1': 4, 'stem2': 32}
lr: 0.003; eff_lr: 0.003; size: 128; alpha: 0.99; mom: 0.9; eps: 1e-06


epoch,train_loss,valid_loss,accuracy,time
0,2.289242,2.296678,0.152174,00:04
1,2.282635,2.290593,0.195652,00:03
2,2.267846,2.275594,0.195652,00:02
3,2.243522,2.242769,0.195652,00:03
4,2.217494,2.190893,0.260870,00:03
5,2.190285,2.132931,0.369565,00:03
6,2.164182,2.078142,0.347826,00:02
7,2.139528,2.027519,0.391304,00:03
8,2.115678,1.984724,0.391304,00:03
9,2.094325,1.954699,0.391304,00:02


this Learner object self-destroyed - it still exists, but no longer usable
{'classes': None, 'sample': 0.25} {'opt': 'sgd', 'stem1': 4, 'stem2': 32}
lr: 0.003; eff_lr: 0.003; size: 128; alpha: 0.99; mom: 0.9; eps: 1e-06


epoch,train_loss,valid_loss,accuracy,time
0,2.304085,2.298427,0.147541,00:07
1,2.277034,2.230657,0.114754,00:04
2,2.237653,2.084227,0.254098,00:06
3,2.192107,1.976423,0.360656,00:05
4,2.147071,1.918309,0.352459,00:05
5,2.106048,1.883777,0.344262,00:07
6,2.069936,1.866512,0.393443,00:05
7,2.034100,1.832659,0.401639,00:05
8,2.001457,1.810174,0.409836,00:07
9,1.969982,1.794942,0.409836,00:04


this Learner object self-destroyed - it still exists, but no longer usable
{'classes': None, 'sample': 0.5} {'opt': 'sgd', 'stem1': 4, 'stem2': 32}
lr: 0.003; eff_lr: 0.003; size: 128; alpha: 0.99; mom: 0.9; eps: 1e-06


epoch,train_loss,valid_loss,accuracy,time
0,2.289698,2.278256,0.148000,00:09
1,2.244272,2.157288,0.296000,00:09
2,2.184474,2.058314,0.324000,00:09
3,2.125891,1.996212,0.336000,00:09
4,2.071764,1.947429,0.352000,00:07
5,2.021364,1.925655,0.368000,00:11
6,1.973531,1.866750,0.396000,00:09
7,1.929482,1.818508,0.412000,00:11
8,1.886142,1.781980,0.420000,00:11
9,1.842360,1.718661,0.448000,00:09


this Learner object self-destroyed - it still exists, but no longer usable
{'classes': None, 'sample': 0.7} {'opt': 'sgd', 'stem1': 4, 'stem2': 32}
lr: 0.003; eff_lr: 0.003; size: 128; alpha: 0.99; mom: 0.9; eps: 1e-06


epoch,train_loss,valid_loss,accuracy,time
0,2.249541,2.159178,0.232143,00:13
1,2.157236,2.001210,0.357143,00:12
2,2.063222,1.911916,0.369048,00:14
3,1.979302,1.853452,0.383929,00:14
4,1.905506,1.804559,0.377976,00:14
5,1.837957,1.758126,0.425595,00:12
6,1.777065,1.695783,0.446429,00:13
7,1.719532,1.678103,0.410714,00:14
8,1.654653,1.556933,0.511905,00:14
9,1.595183,1.507583,0.532738,00:13


this Learner object self-destroyed - it still exists, but no longer usable
{'classes': None, 'sample': 1.0} {'opt': 'sgd', 'stem1': 4, 'stem2': 32}
lr: 0.003; eff_lr: 0.003; size: 128; alpha: 0.99; mom: 0.9; eps: 1e-06


epoch,train_loss,valid_loss,accuracy,time
0,2.249184,2.149946,0.250000,00:15
1,2.129526,1.962733,0.350000,00:16
2,2.003428,1.842073,0.404000,00:15
3,1.894358,1.745781,0.448000,00:16
4,1.795384,1.696150,0.444000,00:16
5,1.717259,1.592628,0.476000,00:16
6,1.639728,1.515710,0.536000,00:16
7,1.573875,1.447485,0.550000,00:16
8,1.513470,1.398570,0.568000,00:18
9,1.465605,1.364986,0.598000,00:16


In [45]:
x in run for x in GRID])

True

In [34]:
STRAT2PARAMS_V2

{'All Classes-0.1': {'classes': None, 'sample': 0.1},
 'All Classes-0.25': {'classes': None, 'sample': 0.25},
 'All Classes-0.5': {'classes': None, 'sample': 0.5},
 'All Classes-0.7': {'classes': None, 'sample': 0.7},
 'All Classes-1.0': {'classes': None, 'sample': 1.0},
 'All Classes-1.0-ep1': {'epochs': 1, 'classes': None, 'sample': 1.0},
 'All Classes-1.0-ep10': {'epochs': 10, 'classes': None, 'sample': 1.0},
 'All Classes-1.0-ep5': {'epochs': 5, 'classes': None, 'sample': 1.0},
 'Half Classes-0.1': {'classes': [0, 1, 2, 3, 4], 'sample': 0.1},
 'Half Classes-0.25': {'classes': [0, 1, 2, 3, 4], 'sample': 0.25},
 'Half Classes-0.5': {'classes': [0, 1, 2, 3, 4], 'sample': 0.5},
 'Half Classes-0.7': {'classes': [0, 1, 2, 3, 4], 'sample': 0.7},
 'Half Classes-1.0': {'classes': [0, 1, 2, 3, 4], 'sample': 1.0},
 'Other Half Classes-1.0': {'classes': [5, 6, 7, 8, 9], 'sample': 1.0},
 'distillation': {},
 'hard-0.0-0.1': {'hardness_lower_bound': 0.0, 'hardness_upper_bound': 0.1},
 'hard-0.0-

In [55]:
STEM_SETTINGS = [
    (SMALL, SMALL), (BIG, BIG),
    (SMALL, 32), (32, SMALL),
    (SMALL, BIG), (BIG, SMALL)
    #(4, 16), (16,4)
]
    
[dict(stem1=a, stem2=b) for (a,b) in STEM_SETTINGS]

[{'stem1': 16, 'stem2': 16},
 {'stem1': 48, 'stem2': 48},
 {'stem1': 16, 'stem2': 32},
 {'stem1': 32, 'stem2': 16},
 {'stem1': 16, 'stem2': 48},
 {'stem1': 48, 'stem2': 16}]

In [50]:
stem_grid = [{'stem1': x} for x in stem_testers]  + [{'stem2': x} for x in stem_testers] + [{'stem1': 16, 'stem2': 16}]
opt_grid = [{'opt': 'SGD', 'opt': 'RMS'}]

In [4]:
%load_ext autoreload
%autoreload 2

In [30]:
from fastai.imagito.grid_const import *

In [6]:
len(NEED_TO_RUN_ERIC_BOX_V4)

48

In [3]:
!git pull

Already up-to-date.


In [51]:
stem_grid

[{'stem1': 16},
 {'stem1': 48},
 {'stem2': 16},
 {'stem2': 48},
 {'stem1': 16, 'stem2': 16}]

In [44]:
df.to_msgpack('/home/paperspace/fastai-fork/fastai/imagito/pred_df_woof.mp')

In [24]:
from fastai.imagito.train_imagito import main

In [ ]:
main(woof=True, classes=range(4))

In [32]:
df

,0
n02115641_44296.JPEG,0.112671
n02115641_7648.JPEG,0.235310
n02088364_1449.JPEG,0.863584
n02099601_8874.JPEG,0.386320
n02089973_4875.JPEG,1.649868
n02099601_447.JPEG,0.195452
n02115641_28502.JPEG,2.505424
n02093754_4460.JPEG,0.003552
n02093754_6370.JPEG,0.300552
n02086240_3672.JPEG,0.000826
